# Building a Boolean Search Engine from Scratch

In this tutorial, we will build a simple boolean search engine from scratch in Python. We will learn how to ...
- ... split text with a tokenizer
- ... parse queries with operators
- ... implement efficient operations on posting lists efficiently
- ... store documents and posting lists in an index
- ... search that index.

Start by installing the required Python library dependencies.

In [ ]:
!pip install -q nltk lark more-itertools

## Preliminaries 1: Tokenizing Text

Our index will later store _tokens_ of each document, to make it searchable.
The _tokenizer_ is responsible for splitting a given text document into tokens (or words). Here, we use NLTK's `MWETokenizer` which you can call as follows:

In [ ]:
from nltk.tokenize import MWETokenizer
tokenizer = MWETokenizer()
print(tokenizer.tokenize("The statue of liberty is the most famous statue in new york".split()))

Currently, the tokenizer does not handle "New York" well. It should not split this term.
As we use a multi-word tokenizer, we can add multi-word tokens through the `add_mwe` function. In the following example, this will result in `new york` being mapped to the token `new_york`.

In [ ]:
tokenizer.add_mwe(("new", "york"))
tokenizer.add_mwe(("statue", "of", "liberty"))
print(tokenizer.tokenize("The statue of liberty is the most famous statue in new york".split()))

## Preliminaries 2: The Query Parser

A boolean query generally consists of a propositional logical formular where the variables are tokens. Formally, we can describe this as a context free grammar:
```
<expr> := TOKEN
       | ( <expr> )
       | <expr> AND <expr>
       | <expr> OR <expr>
       | <expr> AND NOT <expr>
```
where `TOKEN`, `AND`, `OR`, and `NOT` are terminals. Here, we already provide a parser for you which you can call using `parser.parse(query)` to get the operation tree for `query`:

In [ ]:
from lark import Lark, Tree, Token, tree
from rich import print as rprint

parser = Lark('''
            ?expression: andexpr | orexpr | andnotexpr | "(" expression ")" | WORD
            andexpr:    expression "AND" expression
            orexpr:     expression "OR" expression
            andnotexpr:    expression "AND NOT" expression

            %import common.WORD   // imports from terminal library
            %ignore " "           // Disregard spaces in text
         ''', start="expression", )

example = parser.parse("hello AND world OR (foo AND NOT x) AND bar")
print(example)
rprint(example)


## Efficient Operations on Posting Lists

### Generators and Iterators
When dealing with large datasets or streams of data, loading everything into memory at once can be inefficient or even impossible. Generators in Python provide an elegant solution to this problem. They allow you to generate items one at a time, on demand. This not only conserves memory but also enables you to process data as it becomes available, making generators invaluable for tasks like reading large files, streaming data, implementing infinite sequences, or (in our case) handling large lists.

You can create a generator using the `yield` keyword. For example:


In [ ]:
def count_up_to(n: int):
    count = 1
    while count <= n:
        yield count
        count += 1

This generator produces numbers one at a time, which you can iterate over in a loop. However, generators do not support random access since elements are not stored. This means that you cannot access a generator's items by their position like you could with a list. Instead, generators can only be *iterated* over in a `for` loop.

In [ ]:
for x in count_up_to(3):
  print(x)


Sometimes we whish to *peek* an iterator, i.e., look at its first value without stepping the iterator. Iterators themselves don't implement this functionality but the `peekable` utility from the `more_itertools` library lets you wrap a generator and peek at the next value without consuming it. For example:

In [ ]:
from more_itertools import peekable

gen = peekable(count_up_to(5))
print(gen.peek())  # Outputs: 1
print(next(gen))   # Outputs: 1
print(next(gen))   # Outputs: 2

A peekable generator also has a handy check if the iterator is empty, by casting it to a boolean.

In [ ]:
from more_itertools import peekable

gen = peekable(count_up_to(3))
if gen:
  print("Not empty.")
else:
  print("Empty")

# Now, consume all elements of the generator.
for x in gen:
  print(x)

if gen:
  print("Not empty.")
else:
  print("Empty")

This makes it possible to inspect upcoming values while still benefiting from the memory efficiency of generators.

### Efficient, Union, Intersect, and Set Difference

With the recap about generators and iterators out of the way, we can use them to efficiently implement the intersection, union, and set-difference of our posting lists:

In [ ]:
from typing import Iterable
from more_itertools import peekable


def intersect_postings(list1: Iterable[int], list2: Iterable[int]) -> Iterable[int]:
  """
  This function takes two **sorted** posting lists and efficiently intersects them.
  Use generators to iterate the intersection and to avoid as much copying as possible.
  """
  raise NotImplementedError()

In [ ]:
def union_postings(list1: Iterable[int], list2: Iterable[int]) -> Iterable[int]:
  """
  This function takes two **sorted** posting lists and efficiently forms their union.
  Use generators to iterate the intersection and to avoid as much copying as possible.
  """
  raise NotImplementedError()

In [ ]:
def setminus_postings(list1: Iterable[int], list2: Iterable[int]) -> Iterable[int]:
  """
  This function takes two **sorted** posting lists and efficiently calculates
  their set difference, i.e. returns all elements in list1 that are not in list2.
  Use generators to iterate the intersection and to avoid as much copying as possible.
  """
  raise NotImplementedError()

### Task 1: Efficient Posting List Operations

Implement `intersect_postings()` and `union_postings()` above efficiently by exploiting the ordered nature of posting lists. You can use the following code to check your implementation. (Leave `setminus_postings()` unmodified for now.)

In [ ]:
print(list(intersect_postings([1, 2, 6, 9, 10, 12, 13], [1, 2, 4, 8])))
print(list(union_postings([1, 2, 6, 9, 10, 12, 13], [1, 2, 4, 8])))
print(list(setminus_postings([1, 2, 6, 9, 10, 12, 13], [1, 2, 4, 12])))

You can check if your implementation is correct by running the cell below.

In [ ]:
if list(intersect_postings([1, 2, 6, 9, 10, 12, 13], [1, 2, 4, 8])) == [1, 2]:
  print("Bravo! The intersection of postings works")
else:
  print("Oops, that did not work.")
if list(union_postings([1, 2, 6, 9, 10, 12, 13], [1, 2, 4, 8])) == [1, 2, 4, 6, 8, 9, 10, 12, 13]:
  print("Superb! The union of postings works")
else:
  print("Oops, that did not work.")
if list(setminus_postings([1, 2, 6, 9, 10, 12, 13], [1, 2, 4, 12])) == [6, 9, 10, 13]:
  print("Nice! The set difference of postings works")
else:
  print("Oops, that did not work.")

## The Index Structure

Now that we have implemented all the necessary "operators" for the posting lists, and have tried out tokenization and query parsing, let's build an index, so that we can store and search documents. But do not worry! We will start step-by-step.

First, just try to understand the dummy `Index` class.
It will have a reference to the tokenizer (i.e., `self._tokenizer`), store the raw documents (i.e., in the `self._documents` attribute), and store the posting lists (i,e., `self._postings`)

In [ ]:
from nltk.tokenize import MWETokenizer


class Index():
  _tokenizer: MWETokenizer()
  _documents: list[str]
  _postings: dict[str, list[int]]

  def __init__(self):
    """
    Initialize a new, empty index.
    """
    self._tokenizer = MWETokenizer()
    self._documents = NotImplemented
    self._postings = NotImplemented
    raise NotImplementedError()

  def _tokenize(self, text: str) -> set[str]:
    """
    Split the given text into its tokens.
    """
    raise NotImplementedError()

  def add_document(self, text: str):
    """
    Add the new document to the document store and the index.
    The raw document is added to the document store,
    and add a pointer (i.e., the document ID) to the appropriate posting lists.
    """
    raise NotImplementedError()

  def _search(self, query_tree: Tree | Token) -> Iterable[int]:
    """
    Search for a pre-parsed query, given as a syntax tree,
    and return the documents IDs of all documents matching the query.
    """
    raise NotImplementedError()

  def search(self, query: str) -> list[str]:
    """
    Search for a boolean text query and return all documents that
    Takes a query in form of a string, parses it as a boolean query and returns
    the content of all documents that fit the query.
    """
    raise NotImplementedError()

  def get_documents(self) -> list[str]:
    return self._documents

  def get_postings(self) -> dict[str, list[int]]:
    return self._postings

### Task 2: Indexing

Implement the `__init__()` and `add_document()` methods for `Index` as described in their doc strings. For example, think about a reasonable way to initialize the lookup of posting lists by tokens (i.e., `self._postings`).
You are free to choose the order of the documents in the posting list.

*Hint:* The posting list does not have to be a `dict` as long as it is compatible with Python dictionaries. E.g., `defaultdict` may be a better choice here.

After indexing, you can check the posting lists with the `get_postings()` function.

In [ ]:
docs = [
    "the statue of liberty is the most famous statue in new york built gustave eiffel",
    "where is new york",
    "who built the empire state building"
]

index = Index()

for doc in docs:
  index.add_document(doc)

print(index.get_postings())

Let us quickly verify if the indexing has worked.

In [ ]:
postings = index.get_postings()
if "eiffel" in postings and postings["eiffel"] == [0]:
  print("Congratulations! The indexing works.")

### Task 3: Search

Now, we would like to search the index by entering some query.
You have probably noticed two search functions in the template code: `_search()` and `search()` (without underscore).
The latter, `search()`, is what you would want as a user: Given a text query, return text documents.
The `_search()` is a simplification where we already have a parsed query tree and just return the matching document IDs.

#### Task 3a

Let us first implement the easier `_search()` method as described in its doc string.

_Hints:_
You can check if the query is a leaf (i.e., has no sub-trees) using the builtin `isinstance()` function (for example: `isinstance("test", str) == True`). A leaf's text is stored in its `leaf.value` attribute. A trees left and right children are accessed like so `left, right = tree.children`. And the operator type is in the `tree.data` attribute.



In [ ]:
tree = parser.parse("gustave AND eiffel")
print(list(index._search(tree)))

#### Task 3b:

You can now implement the public `search()` function that directly takes a string as the query and returns the documents (as opposed to just the IDs).

_Hint:_ Try to re-use your `_search()` function here.

In [ ]:
print(index.search("gustave AND eiffel"))
print(index.search("(new AND york) OR built"))
print(index.search("new AND (york OR built)"))

### Task 4: The AND NOT Operator

Modify the code above to implement `AND NOT`. Why do you think, don't we implement a general `NOT` operator?

In [ ]:
print(index.search("((new AND york) OR built) AND NOT liberty"))

### Task 5: Multi-Word Tokenization

Update the indexer to tokenize `new york` and `statue of liberty` as a single token each. What would a query requesting information on the Statue of Liberty in New York now look like?

In [ ]:
# TODO: formulate a query requesting information about new york and the statue of liberty
print(index.search(""))

### Task 6 (Bonus): Suffix Wildcards

Update the indexer to support suffix wildcards.

The suffix wildcard, `*`, matches any sequence of characters (even the empty sequence). For example, the pattern `he*` should match, among others, the tokens `he`, `hello`, and `hey`. What part of the indexer have to be updated? What data structure is best suited to find matching patterns?

In [ ]:
print(index.search("wh*"))